In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer, SimpleImputer
from scipy.stats import loguniform, uniform

## a bit of feature engineering
- drop name
- create GroupSize from PassengerID
- create Deck and Side from Cabin
- one hot endoding categorical columns
- scale numerical columns
- HistGB (which accepts nan natively)

In [2]:
PATH = '../data/raw'
df = pd.read_csv(os.path.join(PATH,'train.csv'))
df.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [3]:
def parse(df):
    return (
        df.assign(
            GroupSize = df['PassengerId'].str.split('_', expand=True)[1].astype(int),
            Deck = df['Cabin'].str.split('/',expand=True)[0],
            Side = df['Cabin'].str.split('/',expand=True)[2]
            )
        .drop(['Name','PassengerId','Cabin'],axis=1)
        )

parse(df).nunique()

HomePlanet         3
CryoSleep          2
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Transported        2
GroupSize          8
Deck               8
Side               2
dtype: int64

In [4]:
df = pd.read_csv(os.path.join(PATH,'train.csv'))
df = parse(df)

y = df.pop('Transported')
X = df

ohe = OneHotEncoder()
scaler = StandardScaler()
hgb = HistGradientBoostingClassifier()


ct = make_column_transformer(
    (ohe, ['HomePlanet','CryoSleep', 'Destination', 'VIP', 'GroupSize', 'Deck','Side']),
    (scaler, make_column_selector(dtype_include=np.number)),
    remainder='drop',
)

pipe = make_pipeline(
    ct, hgb
)

param_distributions = {
    'histgradientboostingclassifier__learning_rate': loguniform(0.01,0.1),
    'histgradientboostingclassifier__l2_regularization': loguniform(0.01,10)
}

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions, 
    n_jobs=-1,
    n_iter=20,
    verbose=True,
    random_state=1123
)

X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2, random_state=1123)

search.fit(X_tr,y_tr)
y_pred = search.predict(X_dev)
print(classification_report(y_pred,y_dev))
print(search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
              precision    recall  f1-score   support

       False       0.79      0.79      0.79       819
        True       0.81      0.82      0.81       920

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739

{'histgradientboostingclassifier__l2_regularization': 4.862211624313255, 'histgradientboostingclassifier__learning_rate': 0.06718886122801755}


In [5]:
df_te = pd.read_csv(os.path.join(PATH,'test.csv'))
df_parsed = parse(df_te)
y_pred = search.predict(df_parsed)
df_te['Transported'] = y_pred
df_sub = df_te[['PassengerId','Transported']]
df_sub.to_csv(os.path.join(PATH,'0.1-xy-submission.csv'), index=False)


In [6]:
# !kaggle competitions submit -c spaceship-titanic -f ../data/raw/0.1-xy-submission.csv -m "HistGB-1"
# !kaggle competitions submissions -c spaceship-titanic